In [ ]:
from dataclasses import dataclass
import pandas as pd
from pathlib import Path

In [ ]:
pg_conn_string = "dbname=wrenchball user=postgres password=postgres host=localhost port=5932"

In [ ]:
@dataclass
class User():
    name: str
    initial: str
    discord_snowflake: str
    role: str

In [ ]:
users = [
    User(name="Andrew", initial="AL", discord_snowflake="194910567188791297", role="admin"),
    User(name="Jack", initial="JH", discord_snowflake="178958252820791296", role="admin"),
    User(name="Michael", initial="MC", discord_snowflake="171102907079327745", role="admin"),
    User(name="Matthew", initial="MR", discord_snowflake="687449838945828925", role="user"),
    User(name="ADwy", initial="AD", discord_snowflake="200403036252798976", role="user"),
    User(name="NDA", initial="NDA", discord_snowflake="89554070678827008", role="user"),
    User(name="Evan", initial="EO", discord_snowflake="478366445995032596", role="user"),
    User(name="Luke", initial="LB", discord_snowflake="158029629683269632", role="user"),
    User(name="Noran", initial="AN", discord_snowflake="262332170041819146", role="user"),
    User(name="Adam", initial="AW", discord_snowflake="606620386225946624", role="user"),
]

In [ ]:
import psycopg

with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for user in users:
            cur.execute(
                """
                INSERT INTO "user" (name, role, discord_snowflake)
                VALUES (%s, %s, %s)
                """,
                (user.name, user.role, user.discord_snowflake)
            )

    conn.commit()

In [ ]:
with psycopg.connect(pg_conn_string) as conn:
    with conn.cursor() as cur:
        for user in users:
            # Fetch user id from the database using discord_snowflake
            cur.execute(
                """
                SELECT id FROM "user" WHERE discord_snowflake = %s
                """,
                (user.discord_snowflake,)
            )
            result = cur.fetchone()
            if result is not None:
                user_id = result[0]
                team_name = f"{user.name}'s Team"
                abbreviation = user.initial
                cur.execute(
                    """
                    INSERT INTO team (name, user_id, abbreviation)
                    VALUES (%s, %s, %s)
                    """,
                    (team_name, user_id, abbreviation)
                )
            else:
                raise ValueError(f"User {user.name} not found in database")
    conn.commit()

In [ ]:
s3_xlsx_path = Path("data/sheets/lil sLUg crew S3.xlsx")

In [ ]:
s3_df = pd.read_excel(s3_xlsx_path, sheet_name="Character List")

s3_df.head()

In [ ]:
players = [c for c in s3_df["Character"].dropna().unique().tolist() if c != 'Empty']

players

In [ ]:
# https://docs.google.com/spreadsheets/d/13uVzRaigr4i-QRZAhp4DcbDuhxhJvwxZigWds_Y8CR4/edit?gid=66049872#gid=66049872

